# 模型测试


SageMaker 目前部署了 4 个模型：


- THUDM/chatglm2-6b LLM 对话模型 （对应 type>3 or type<0）
- shibing624/text2vec-base-chinese 向量转化模型（对应 type=0）
- shibing624/text2vec-base-chinese-sentence 向量转化模型（对应 type=1）
- sentence-transformers/paraphrase-multilingual-mpnet-base-v2 向量转化模型（对应 type=2）

In [7]:
from sagemaker.huggingface.model import HuggingFacePredictor

from numpy import dot
from numpy.linalg import norm

def numpy_sim(v1, v2):
    return dot(v1, v2)/(norm(v1)*norm(v2))

predictor = HuggingFacePredictor(
  endpoint_name='llm-models'
)


## LLM 对话模型

In [10]:
answer = predictor.predict({
    "text": f"""
    翻译下面的文档为莎士比亚风格的英文：

    日照香炉生紫烟，遥望瀑布挂前川
    """,
    "type": 8
})

print(answer)

The sun rises, fragrant smoke ascending

As I gaze at the falls, far ahead

In the distance, a waterfall cascades

And the air is filled with the sweet aroma


In [12]:
answer = predictor.predict({
    "text": f"""
    Translate the following text into Chinese:

    The sun sets over the mountains, and the Yellow River flows into the sea.
      """,
    "type": 8
})

print(answer)

落日落在山脉上，黄河流入大海。


## 向量提取

In [13]:
res = predictor.predict({
    "text": "你好世界",
    "type": 2
})

print(len(res), res[:10])

768 [0.03347698226571083, 0.1332712173461914, -0.013817096129059792, 0.06620144098997116, 0.1956453025341034, 0.006277161650359631, -0.011438372544944286, 0.022746816277503967, 0.06149399280548096, 0.026750221848487854]


## 向量相似性测试

### 数据准备

In [ ]:
word = "对不起"
compares = ["抱歉", "對不起", "對唔住","真遗憾", "吃了吗", "I'm sorry", "I apologize", "the weather",  "ごめんなさい", "미안해", "Désolé"]


### 语义相似性 shibing624/text2vec-base-chinese 

In [ ]:

wordvec = predictor.predict({
    "text": word,
    "type": 0
})


for compare in compares:
  print(f"{word} 与 {compare} 的相似度：",  numpy_sim(predictor.predict({
      "text": compare,
      "type": 0
  }), wordvec))

### 语义相似性 shibing624/text2vec-base-chinese-sentence

In [ ]:

wordvec = predictor.predict({
    "text": word,
    "type": 1
})


for compare in compares:
  print(f"{word} 与 {compare} 的相似度：",  numpy_sim(predictor.predict({
      "text": compare,
      "type": 1
  }), wordvec))

### 语义相似性 sentence-transformers/paraphrase-multilingual-mpnet-base-v2

In [ ]:

wordvec = predictor.predict({
    "text": word,
    "type": 2
})


for compare in compares:
  print(f"{word} 与 {compare} 的相似度：",  numpy_sim(predictor.predict({
      "text": compare,
      "type": 2
  }), wordvec))

### 语义相似性 ChatGLM2 embeddings

In [ ]:

wordvec = predictor.predict({
    "text": word,
    "type": 3
})


for compare in compares:
  print(f"{word} 与 {compare} 的相似度：",  numpy_sim(predictor.predict({
      "text": compare,
      "type": 3
  }), wordvec))

In [ ]:
vec1 =  predictor.predict({
    "text": "在校生人数排在全国第三位的",
    "type": 2
})

vec2 = predictor.predict({
    "text": "山东大学，在校生人数排在全国第三位的，无明显特色，靠庞大的“综合性”支撑门面招牌。 理科最好的专业：数学、材料工程、物理、机械制造、控制工程、医学等 文科最好的专业：经济学、文学、历史学等",
    "type": 2
})
vec3 = predictor.predict({
    "text": "1、华东师范大学，如果说北京师大是中国师范第一名，那么第二名非华东师范大学莫属。 自然就形成了南北鼎立的状态，北师大统一北方，华东师大则领衔南方。 理科最好的专业：地理学、数学、光学、生命科学 文科最好的专业：教育学、文学、心理学",
    "type": 2
})
print(numpy_sim(vec1, vec2))
print(numpy_sim(vec1, vec3))

In [ ]:
vec1 =  predictor.predict({
    "text": "华东师范大学",
    "type": 1
})

vec2 = predictor.predict({
    "text": "山东大学，在校生人数排在全国第三位的，无明显特色，靠庞大的“综合性”支撑门面招牌。 理科最好的专业：数学、材料工程、物理、机械制造、控制工程、医学等 文科最好的专业：经济学、文学、历史学等",
    "type": 1
})
vec3 = predictor.predict({
    "text": "1、华东师范大学，如果说北京师大是中国师范第一名，那么第二名非华东师范大学莫属。 自然就形成了南北鼎立的状态，北师大统一北方，华东师大则领衔南方。 理科最好的专业：地理学、数学、光学、生命科学 文科最好的专业：教育学、文学、心理学",
    "type": 1
})
print(numpy_sim(vec1, vec2))
print(numpy_sim(vec1, vec3))

In [ ]:
vec1 =  predictor.predict({
    "text": "华东师范大学",
    "type": 0
})

vec2 = predictor.predict({
    "text": "山东大学，在校生人数排在全国第三位的，无明显特色，靠庞大的“综合性”支撑门面招牌。 理科最好的专业：数学、材料工程、物理、机械制造、控制工程、医学等 文科最好的专业：经济学、文学、历史学等",
    "type": 0
})
vec3 = predictor.predict({
    "text": "1、华东师范大学，如果说北京师大是中国师范第一名，那么第二名非华东师范大学莫属。 自然就形成了南北鼎立的状态，北师大统一北方，华东师大则领衔南方。 理科最好的专业：地理学、数学、光学、生命科学 文科最好的专业：教育学、文学、心理学",
    "type": 0
})
print(numpy_sim(vec1, vec2))
print(numpy_sim(vec1, vec3))

In [ ]:
vec1 =  predictor.predict({
    "text": "university in south",
    "type": 2
})

vec2 = predictor.predict({
    "text": "山东大学，在校生人数排在全国第三位的，无明显特色，靠庞大的“综合性”支撑门面招牌。 理科最好的专业：数学、材料工程、物理、机械制造、控制工程、医学等 文科最好的专业：经济学、文学、历史学等",
    "type": 2
})
vec3 = predictor.predict({
    "text": "1、华东师范大学，如果说北京师大是中国师范第一名，那么第二名非华东师范大学莫属。 自然就形成了南北鼎立的状态，北师大统一北方，华东师大则领衔南方。 理科最好的专业：地理学、数学、光学、生命科学 文科最好的专业：教育学、文学、心理学",
    "type": 2
})
print(numpy_sim(vec1, vec2))
print(numpy_sim(vec1, vec3))